In [1]:
import random
import pandas as pd
import numpy as np

In [2]:
# Hyperparameters
D_VOCAB = 64

In [20]:
def sorted_list_random_numbers(length):
    return sorted([random.randint(0, D_VOCAB) for i in range(length)])

def displace(lst):
    i, j = 0,0
    while abs(i - j) < 1:
        i = random.randint(0, len(lst)-1)
        j = random.randint(0, len(lst)-1)
    if i > j:
        new_lst = lst[:j]
        new_lst.append(lst[i])
        new_lst += lst[j:i]
        new_lst += lst[i+1:]
    else: # j < i
        new_lst = lst[:i]
        new_lst += lst[i+1:j]
        new_lst.append(lst[i])
        new_lst += lst[j:]
    return new_lst

def generate_swap_dataset_1(length=4, num_samples = 2000):
    sorted_lst = [sorted_list_random_numbers(length) for i in range(num_samples)]
    df = pd.DataFrame()
    df["sorted"] = sorted_lst
    df["displaced"] = [displace(lst) for lst in sorted_lst]
    return df

generate_swap_dataset_1()

,sorted,displaced
0,"[11, 16, 23, 46]","[16, 11, 23, 46]"
1,"[0, 4, 17, 45]","[4, 0, 17, 45]"
2,"[0, 53, 56, 64]","[53, 0, 56, 64]"
3,"[28, 35, 35, 48]","[35, 28, 35, 48]"
4,"[17, 42, 57, 58]","[17, 42, 57, 58]"
...,...,...
1995,"[12, 24, 51, 62]","[12, 51, 24, 62]"
1996,"[2, 8, 13, 52]","[13, 2, 8, 52]"
1997,"[6, 23, 31, 35]","[23, 6, 31, 35]"
1998,"[9, 30, 47, 54]","[30, 47, 9, 54]"


# Parse to CSV file

Takes the Data Frame and parses it to a csv file of the format:

text, labels
70 158 307 263 [SEP] [MASK] [MASK] [MASK] [MASK], 70 158 263 307


In [21]:
def parse_inputs(row):
    lst = row[0]
    pattern = ""

    # Unpack the list into a string of values
    for i in lst:
        pattern += f"{i} "

    # Add the remaining part
    pattern += "[SEP] [MASK] [MASK] [MASK] [MASK]"    
    return pattern

def parse_outputs(row):
    lst = row[1]
    pattern = ""

    # Unpack the list into a string of values
    for i in lst:
        pattern += f"{i} "
    return pattern

def parse_data_to_csv(data, file_name):
    data["text"] = data.apply(parse_inputs, axis=1)
    data["labels"] = data.apply(parse_outputs, axis=1)

    # Write the DataFrame to a CSV file
    data.loc[:,["text", "labels"]].to_csv(file_name, index=False)

In [22]:
data = generate_swap_dataset_1()
print(data)
parse_data_to_csv(data, "../data/displace_test.csv")

                sorted         displaced
0       [0, 8, 31, 59]    [0, 59, 8, 31]
1     [10, 20, 21, 39]  [20, 10, 21, 39]
2      [4, 20, 39, 64]   [4, 39, 20, 64]
3     [23, 28, 37, 37]  [23, 28, 37, 37]
4      [9, 38, 54, 61]   [54, 9, 38, 61]
...                ...               ...
1995   [1, 23, 42, 50]   [1, 23, 42, 50]
1996  [18, 20, 38, 48]  [18, 20, 38, 48]
1997  [28, 40, 47, 55]  [28, 40, 47, 55]
1998    [3, 3, 45, 62]    [3, 3, 62, 45]
1999  [10, 37, 39, 39]  [37, 39, 10, 39]

[2000 rows x 2 columns]
